In [91]:
from tqdm import tqdm

from random import randint

In [2]:
def parse_input(file):
    with open(file) as file_in:
        input_str = file_in.read()

    registers, program = input_str.split('\n\n')
    registers = [rule.split(': ') for rule in registers.splitlines()]
    registers = {id.split(' ')[1]: int(init_value) for id, init_value in registers}
    program = program.split(': ')[1][:-1].split(',')
    program = tuple([int(n) for n in program])

    return program, registers

In [3]:
def compute_combo_operand(operand, registers):
    if 0 <= operand <= 3:
        return operand
    elif operand == 4:
        return registers['A']
    elif operand == 5:
        return registers['B']
    elif operand == 6:
        return registers['C']
    else:
        raise ValueError(f'Operand {operand} is invalid')

In [4]:
def run_program(program, registers):
    ip = 0
    output = []
    while ip < len(program):
        opcode, operand = program[ip], program[ip+1]
        if opcode == 0:
            # adv
            registers['A'] = int(registers['A'] / 2**compute_combo_operand(operand, registers))
        elif opcode == 1:
            # bxl
            registers['B'] = registers['B'] ^ operand
        elif opcode == 2:
            # bst
            registers['B'] = compute_combo_operand(operand, registers) % 8
        elif opcode == 3:
            # jnz
            if registers['A'] != 0:
                ip = operand
                continue
        elif opcode == 4:
            # bxc
            registers['B'] = registers['B'] ^ registers['C']
        elif opcode == 5:
            # out
            out = compute_combo_operand(operand, registers) % 8
            output.append(out)
        elif opcode == 6:
            # bdv
            registers['B'] = int(registers['A'] / 2**compute_combo_operand(operand, registers))
        elif opcode == 7:
            # bdv
            registers['C'] = int(registers['A'] / 2**compute_combo_operand(operand, registers))
        ip += 2

        output_str = ','.join(map(str, output))

    return output_str

In [5]:
def main1(file):
    program, registers = parse_input(file)
    output = run_program(program, registers)
    return output

In [6]:
assert main1('example1.txt') == '4,6,3,5,6,3,5,2,1,0'

In [7]:
main1('input.txt')

'6,0,6,3,0,2,3,1,6'

In [8]:
def main2(file):
    program, registers = parse_input(file)
    program_str = ','.join(map(str, program))
    for i in range(int(1e9)):
        registers_tmp = registers.copy()
        registers_tmp['A'] = i
        if run_program(program, registers_tmp) == program_str:
            return i

In [84]:
def reverse_engineer_na_exp2():
    program, __ = parse_input('example2.txt')
    program_rev = program[::-1]

    n_steps = len(program)

    sup = int(8**n_steps)
    inf = int(8**(n_steps-1))

    na_bin = []
    for i in range(n_steps):
        na_bin.append(f'{program_rev[i]:03b}')

    return int(''.join(na_bin), 2) * 8

In [85]:
assert reverse_engineer_na_exp2() == 117440

In [98]:
program, __ = parse_input('input.txt')
program_rev = program[::-1]

n_steps = len(program) - 1

sup = int(8**n_steps)
inf = int(8**(n_steps-1))

na = randint(inf, sup)
for i in range(n_steps+1):
    if i >= 1:
        na = int(na / 8)
    out = int(na / 2**((na % 8) ^ 3))
    print(i, na, out, f'{na:b}')

0 17311843489162 8655921744581 11111011111010111010010010000001010110001010
1 2163980436145 540995109036 11111011111010111010010010000001010110001
2 270497554518 8453048578 11111011111010111010010010000001010110
3 33812194314 16906097157 11111011111010111010010010000001010
4 4226524289 1056631072 11111011111010111010010010000001
5 528315536 66039442 11111011111010111010010010000
6 66039442 33019721 11111011111010111010010010
7 8254930 4127465 11111011111010111010010
8 1031866 515933 11111011111010111010
9 128983 8061 11111011111010111
10 16122 8061 11111011111010
11 2015 125 11111011111
12 251 251 11111011
13 31 1 11111
14 3 3 11
15 0 0 0
